<a href="https://colab.research.google.com/github/rahiakela/automl-experiments/blob/main/automl-in-action/04-end-to-end-ml-solutions/01_automated_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Addressing tasks with multiple inputs or outputs

An ML task could involve multiple inputs collected from different resources, which we
call different data modalities.

For example, images could be associated with tags and
other text descriptions, and videos could contain both visual and acoustic information
(as well as metadata) that is useful for classification.

Multiple inputs augment information
resources. They could benefit and compensate with each other to help train a
better ML model. This approach is called multi-input learning or multimodal learning.

Similarly, we might want to have multiple outputs corresponding to different tasks
(regression or classification) that we address simultaneously. This is called multi-output
learning or multitask learning.


##Setup

In [ ]:
!pip install autokeras==1.0.18
!pip install keras-tuner==1.1.0

In [4]:
from tensorflow.keras.datasets import mnist
import autokeras as ak

##Automated image classification

The IO API can generalize to all types of data and tasks, so
we need to provide information about the data types and task types during initialization,
so it can select the appropriate loss function, metrics, search space, and
search objective.

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Training image shape:", x_train.shape)  # (60000, 28, 28)
print("Training label shape:", y_train.shape)  # (60000,)
print("First five training labels:", y_train[:5])  # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


In [7]:
# Initialize the IO API
io_model = ak.AutoModel(
    inputs=ak.ImageInput(),
    outputs=ak.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
    objective="val_loss",
    tuner="random",
    max_trials=3,
    overwrite=True
)

# Fit the model with prepared data
io_model.fit(x_train[:100], y_train[:100], epochs=1)
# Use the first 100 training samples for 1 epoch as a quick demo.
# You may run with the full dataset with 10 epochs, but expect a longer training time.

Trial 3 Complete [00h 00m 48s]
val_loss: 2.30248761177063

Best val_loss So Far: 2.30248761177063
Total elapsed time: 00h 01m 19s
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 27s 5s/step - loss: 2.3511 - accuracy: 0.1200
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [8]:
# Get the summarized results during the tuning process
io_model.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
image_block_1/normalize: True
image_block_1/augment: False
image_block_1/block_type: xception
classification_head_1/spatial_reduction_1/reduction_type: global_avg
classification_head_1/dropout: 0.5
optimizer: adam
learning_rate: 1e-05
image_block_1/xception_block_1/pretrained: False
image_block_1/xception_block_1/imagenet_size: False
Score: 2.30248761177063
Trial summary
Hyperparameters:
image_block_1/normalize: True
image_block_1/augment: True
image_block_1/block_type: vanilla
classification_head_1/spatial_reduction_1/reduction_type: global_avg
classification_head_1/dropout: 0.5
optimizer: adam
learning_rate: 2e-05
image_block_1/image_augmentation_1/translation_factor: 0.0
image_block_1/image_augmentation_1/horizontal_flip: True
image_block_1/image_augmentation_1/vertical_flip: True
image_block_1/image_augmentation_1/rotation_factor: 0.0
image_block

In [9]:
# Retrieve best model
best_model = io_model.export_model()
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 cast_to_float32 (CastToFloat32  (None, 28, 28)      0           ['input_1[0][0]']                
 )                                                                                                
                                                                                                  
 expand_last_dim (ExpandLastDim  (None, 28, 28, 1)   0           ['cast_to_float32[0][0]']        
 )                                                                                                
                                                                                              

In [10]:
# Predict with the best model
predicted_y = io_model.predict(x_test[:100])
print(predicted_y)

4/4 [==============================] - 2s 521ms/step
[['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['1']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['9']
 ['1']
 ['9']
 ['9']
 ['9']]


In [11]:
# Evaluate the best model on the test data
test_loss, test_acc = io_model.evaluate(x_test[:100], y_test[:100])
print("Test accuracy: ", test_acc)

4/4 [==============================] - 3s 514ms/step - loss: 2.3025 - accuracy: 0.1200
Test accuracy:  0.11999999731779099
